In [2]:
import getpass
import os

if not os.environ.get("DEEPSEEK_API_KEY"):
  os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter API key for DeepSeek: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("deepseek-chat", model_provider="deepseek")

In [3]:
!echo $DEEPSEEK_API_KEY

sk-e0d5ecda6bb545cd9584fbeba3c11702


In [4]:
llm

ChatDeepSeek(client=<openai.resources.chat.completions.completions.Completions object at 0x10ad32270>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10ad9cc50>, root_client=<openai.OpenAI object at 0x10799a5d0>, root_async_client=<openai.AsyncOpenAI object at 0x10ad9cbf0>, model_name='deepseek-chat', model_kwargs={}, api_key=SecretStr('**********'), api_base='https://api.deepseek.com/v1')

In [5]:
# from sentence_transformers import SentenceTransformer
# from sentence_transformers.util import cos_sim

# sentences = ['那是一个快乐的人', '那是一个非常快乐的人']

# model = SentenceTransformer('thenlper/gte-large-zh')
# embeddings = model.encode(sentences)
# print(cos_sim(embeddings[0], embeddings[1]))


In [6]:
# len(embeddings[0])

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-large-zh")

/Users/ethanliu/miniconda3/envs/BidMaster_p12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_milvus import Milvus

URI = "./milvus_example.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
    auto_id=True
)

In [9]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/ethanliu/miniconda3/envs/BidMaster_p12/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
response = graph.invoke({"question": "什么是任务分解？"})
print(response["answer"])

任务分解是将复杂任务拆解为多个更小、更简单的步骤的过程。例如，思维链（CoT）技术通过“逐步思考”将大任务分解为多个可管理的子任务。这种方法有助于模型更清晰地处理复杂问题。  

（注：根据上下文，任务分解在AI领域常通过CoT等技术实现，但未直接定义“任务分解”一词，故基于相关描述提炼核心概念。）
